In [3]:
import glob
import os
import random
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np

In [4]:
class CUAV_Model:
    
    def __init__(self):
        self.n_hidden = 40
        pass

    def windows(self, data, window_size):
        start = 0
        while start < len(data):
            yield start, start + window_size
            start += (window_size / 2)

    # 각 33개 input, 각 6개 label

    def extract_features(self, file_path, file_label, file_ext="*.wav",bands = 20, frames = 41):
        window_size = 512 * (frames - 1)
        mfccs = []
        log_specgrams = []
        features = []
        labels = []
        sound_clip, s = librosa.load(file_path)
        #print(type(sound_clip))
        if file_label=='other':
            label_code = 0
        elif file_label=='person':
            label_code = 1
        elif file_label=='car':
            label_code = 2
        elif file_label=='drone':
            label_code = 3

        #print(file_label, label_code)
        for (start,end) in self.windows(sound_clip,window_size):
            start = int(start)
            end = int(end)
            if(len(sound_clip[start:end]) == window_size):
                signal = sound_clip[start:end]

                melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                logspec = librosa.amplitude_to_db(melspec)
                logspec = logspec.T.flatten()[:, np.newaxis].T
                #print(1, logspec.shape)
                log_specgrams.append(logspec)

                mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                mfccs.append(mfcc)
                #print(2, mfcc.shape)
                features = np.hstack((mfccs, log_specgrams))
                labels.append(label_code)         
        features = np.asarray(features).reshape(len(mfccs), frames, bands*2)
        #print(features.shape)
        return np.array(features), np.array(labels,dtype = np.int)

    def extract_features_for_predict(self, file_path, bands = 20, frames = 41):
        window_size = 512 * (frames - 1)
        mfccs = []
        log_specgrams = []
        features = []
        sound_clip, s = librosa.load(file_path)

        for (start,end) in self.windows(sound_clip,window_size):
            start = int(start)
            end = int(end)
            if(len(sound_clip[start:end]) == window_size):
                signal = sound_clip[start:end]

                melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                logspec = librosa.amplitude_to_db(melspec)
                logspec = logspec.T.flatten()[:, np.newaxis].T
                log_specgrams.append(logspec)

                mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                mfccs.append(mfcc)
                features = np.hstack((mfccs, log_specgrams))      

        features = np.asarray(features).reshape(len(mfccs), frames, bands*2)
        #print(features.shape)
        return np.array(features)

    def one_hot_encode(self, labels):
        n_labels = len(labels)
        n_unique_labels = len(np.unique(labels))
        one_hot_encode = np.zeros((n_labels,n_unique_labels))
        one_hot_encode[np.arange(n_labels), labels] = 1
        return one_hot_encode
    
    def make_data(self):
        wav_file_path_training = 'C://slice_wav_data/training/'
        file_list_training = os.listdir(wav_file_path_training)

        tr_features = []
        tr_labels = []
        for f in file_list_training:
            file_label = f.split("_")[0]
            #if file_label=='person' or file_label=='car': ## 2개씩만 하는 코드
             #   continue
            features_temp, labels_temp = self.extract_features(wav_file_path_training + f, file_label)
            for tr_f in features_temp:
                tr_features.append(tr_f)
            for tr_l in labels_temp:
                tr_labels.append(tr_l)


        tmp = [[x,y] for x,y in zip(tr_features, tr_labels)]
        random.shuffle(tmp)
        tr_features = [n[0] for n in tmp]
        tr_labels = [n[1] for n in tmp]


        wav_file_path_test = 'C://slice_wav_data/testing/'
        file_list_test = os.listdir(wav_file_path_test)

        ts_features = []
        ts_labels = []
        for f in file_list_test:
            file_label = f.split("_")[0]
            #if file_label=='person' or file_label=='car': ## 2개씩만 하는 코드
             #   continue
            features_temp, labels_temp = self.extract_features(wav_file_path_test + f, file_label)
            for ts_f in features_temp:
                ts_features.append(ts_f)
            for ts_l in labels_temp:
                ts_labels.append(ts_l)

        tr_labels = self.one_hot_encode(tr_labels)
        ts_labels = self.one_hot_encode(ts_labels)

        self.tr_features = np.array(tr_features)
        self.tr_labels = np.array(tr_labels)

        self.ts_features = np.array(ts_features)
        self.ts_labels = np.array(ts_labels)
        
        
    def RNN(self, x, weight, bias):
        cell = rnn_cell.LSTMCell(self.n_hidden,state_is_tuple = True)
        cell = rnn_cell.MultiRNNCell([cell] * 8, state_is_tuple=True)
        output, state = tf.nn.dynamic_rnn(cell, x, dtype = tf.float32)
        output = tf.transpose(output, [1, 0, 2])
        last = tf.gather(output, int(output.get_shape()[0]) - 1)
        return tf.nn.softmax(tf.matmul(last, weight) + bias)
    
    def graph_setting(self):
        tf.reset_default_graph()
        self.session = tf.Session()

        learning_rate = 0.0003

        # Network Parameters
        n_input = 40
        n_steps = 41
        n_classes = 4

        #앞에거는 hidden *2, 뒤에거는 n_input + n_hidden

        self.x = tf.placeholder("float", [None, n_steps, n_input])
        self.y = tf.placeholder("float", [None, n_classes])

        weight = tf.Variable(tf.random_normal([self.n_hidden, n_classes]))
        bias = tf.Variable(tf.random_normal([n_classes]))
        
        self.prediction = self.RNN(self.x, weight, bias)
        self.prediction_str = tf.argmax(self.prediction, 1)
        #prediction_str, prediction, x, 
        # Define loss and optimizer
        self.loss_f = -tf.reduce_sum(self.y * tf.log(self.prediction))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.loss_f)

        # Evaluate model
        self.correct_pred = tf.equal(tf.argmax(self.prediction,1), tf.argmax(self.y,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))

        # Initializing the variables
        
    def training(self):
        init = tf.global_variables_initializer()
        self.session.run(init)

        training_epochs = 10000
        batch_size = 54 #1188과 216의 최대공약수는 54
        
        for epoch in range(training_epochs):
            avg_cost = 0
            total_batch = int(len(self.tr_features) / batch_size)

            for i in range(total_batch):
                start = ((i+1) * batch_size) - batch_size
                end = ((i+1) * batch_size)
                batch_x = self.tr_features[start:end]
                batch_y = self.tr_labels[start:end]

                _, c = self.session.run([self.optimizer, self.loss_f], feed_dict={self.x: batch_x, self.y : batch_y})
                avg_cost += c / total_batch

            print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), end='')
            print('Test accuracy: ',round(self.session.run(self.accuracy, feed_dict={self.x: self.ts_features, self.y: self.ts_labels}) , 3))
            if epoch % 100 == 0:
                self.save_network(epoch)
        
        print('Learning Finished!')

        
    def save_network(self, step):
        saver = tf.train.Saver()
        saver.save(self.session, './rnn_graph_save_3/cuav_rnn.ckpt', step)
        print('Graph Saved! ')
        
    def predict(self, file_source):
        print(file_source)
        data_to_predict = self.extract_features_for_predict(file_source)
        print(data_to_predict.shape)
        result_list = self.session.run(self.prediction_str, feed_dict={self.x: data_to_predict})
        print(result_list)
        result_list = list(result_list)
        label_count = {'others': 0, 'person': 0, 'car': 0, 'drone': 0,}

        total = 0
        for i, key in enumerate(list(label_count.keys())):
            label_count[key] = (result_list.count(i))
            print(key, int(label_count[key])/data_to_predict.shape[0])

        t = list(zip(list(label_count.values()), list(label_count.keys())))
        t.sort(reverse=True)
        print(t)
        print("")

        #for i in range(data_to_predict.shape[0]):
            #temp_output = session.run(prediction, feed_dict={x: data_to_predict[i]})
            #print(temp_output)
        
    def restore_graph(self, step):
        save_file = './rnn_graph_save/cuav_rnn.ckpt' + '-' + str(step)
        saver = tf.train.Saver()
        saver.restore(self.session, save_file)
        

        #sess = tf.InteractiveSession()
        #saver = tf.train.import_meta_graph('./cuav_rnn.ckpt.meta')
        #saver.restore(self.sess,'./cuav_rnn.ckpt.ckpt')

        #graph = tf.get_default_graph()
        #self.X =  self.sess.graph.get_tensor_by_name("Placeholder:0")
        #self.Y =  self.sess.graph.get_tensor_by_name("Placeholder_1:0")
    

In [5]:
m1 = CUAV_Model()

In [6]:
m1.make_data()

In [7]:
m1.graph_setting()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


C:\Users\kvlks\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [8]:
m1.training()

Epoch: 0001 cost = 79.889763225Test accuracy:  0.231
Graph Saved! 
Epoch: 0002 cost = 74.810160203Test accuracy:  0.333
Epoch: 0003 cost = 73.211332148Test accuracy:  0.329
Epoch: 0004 cost = 71.028516249Test accuracy:  0.319
Epoch: 0005 cost = 69.643386147Test accuracy:  0.31
Epoch: 0006 cost = 64.630699678Test accuracy:  0.301
Epoch: 0007 cost = 54.826515198Test accuracy:  0.384
Epoch: 0008 cost = 47.778036291Test accuracy:  0.444
Epoch: 0009 cost = 42.937593980Test accuracy:  0.481
Epoch: 0010 cost = 36.615124009Test accuracy:  0.509
Epoch: 0011 cost = 36.033992594Test accuracy:  0.468
Epoch: 0012 cost = 33.929876674Test accuracy:  0.5
Epoch: 0013 cost = 33.080239209Test accuracy:  0.519
Epoch: 0014 cost = 32.720843749Test accuracy:  0.602
Epoch: 0015 cost = 31.511805621Test accuracy:  0.593
Epoch: 0016 cost = 29.666879047Test accuracy:  0.602
Epoch: 0017 cost = 27.570323164Test accuracy:  0.611
Epoch: 0018 cost = 26.992025982Test accuracy:  0.574
Epoch: 0019 cost = 27.328248804Test

Epoch: 0156 cost = 10.397126653Test accuracy:  0.657
Epoch: 0157 cost = 13.796976144Test accuracy:  0.634
Epoch: 0158 cost = 13.105141661Test accuracy:  0.657
Epoch: 0159 cost = 10.382960385Test accuracy:  0.671
Epoch: 0160 cost = 7.920058576Test accuracy:  0.69
Epoch: 0161 cost = 8.366146933Test accuracy:  0.708
Epoch: 0162 cost = 8.042222522Test accuracy:  0.718
Epoch: 0163 cost = 7.534803336Test accuracy:  0.644
Epoch: 0164 cost = 9.465900183Test accuracy:  0.704
Epoch: 0165 cost = 9.022257003Test accuracy:  0.671
Epoch: 0166 cost = 7.810673454Test accuracy:  0.69
Epoch: 0167 cost = 7.768551426Test accuracy:  0.644
Epoch: 0168 cost = 7.369365465Test accuracy:  0.708
Epoch: 0169 cost = 6.813953378Test accuracy:  0.671
Epoch: 0170 cost = 7.332888831Test accuracy:  0.639
Epoch: 0171 cost = 8.705070301Test accuracy:  0.625
Epoch: 0172 cost = 7.956911456Test accuracy:  0.699
Epoch: 0173 cost = 7.567746693Test accuracy:  0.671
Epoch: 0174 cost = 7.022921497Test accuracy:  0.694
Epoch: 017

Epoch: 0313 cost = 5.401063832Test accuracy:  0.676
Epoch: 0314 cost = 5.077631365Test accuracy:  0.69
Epoch: 0315 cost = 7.405946851Test accuracy:  0.704
Epoch: 0316 cost = 6.651380940Test accuracy:  0.681
Epoch: 0317 cost = 4.998158867Test accuracy:  0.704
Epoch: 0318 cost = 3.739928814Test accuracy:  0.731
Epoch: 0319 cost = 3.833969533Test accuracy:  0.685
Epoch: 0320 cost = 3.768488892Test accuracy:  0.727
Epoch: 0321 cost = 3.326111252Test accuracy:  0.713
Epoch: 0322 cost = 4.266811742Test accuracy:  0.704
Epoch: 0323 cost = 4.366293758Test accuracy:  0.722
Epoch: 0324 cost = 3.926362997Test accuracy:  0.708
Epoch: 0325 cost = 3.279962025Test accuracy:  0.718
Epoch: 0326 cost = 3.582291205Test accuracy:  0.676
Epoch: 0327 cost = 4.502785967Test accuracy:  0.699
Epoch: 0328 cost = 5.241528319Test accuracy:  0.704
Epoch: 0329 cost = 6.038862293Test accuracy:  0.639
Epoch: 0330 cost = 4.085351066Test accuracy:  0.704
Epoch: 0331 cost = 4.885834366Test accuracy:  0.685
Epoch: 0332 c

Epoch: 0471 cost = 3.838029836Test accuracy:  0.731
Epoch: 0472 cost = 5.123118707Test accuracy:  0.718
Epoch: 0473 cost = 3.055252349Test accuracy:  0.704
Epoch: 0474 cost = 2.024444485Test accuracy:  0.69
Epoch: 0475 cost = 3.250902146Test accuracy:  0.676
Epoch: 0476 cost = 2.376603709Test accuracy:  0.704
Epoch: 0477 cost = 3.304832507Test accuracy:  0.75
Epoch: 0478 cost = 2.507235448Test accuracy:  0.727
Epoch: 0479 cost = 2.886225232Test accuracy:  0.681
Epoch: 0480 cost = 2.629115564Test accuracy:  0.694
Epoch: 0481 cost = 2.536971351Test accuracy:  0.685
Epoch: 0482 cost = 6.092707106Test accuracy:  0.685
Epoch: 0483 cost = 8.922105163Test accuracy:  0.731
Epoch: 0484 cost = 18.188699365Test accuracy:  0.699
Epoch: 0485 cost = 7.734427506Test accuracy:  0.718
Epoch: 0486 cost = 3.718573565Test accuracy:  0.713
Epoch: 0487 cost = 2.795402829Test accuracy:  0.727
Epoch: 0488 cost = 2.803402722Test accuracy:  0.704
Epoch: 0489 cost = 2.292918059Test accuracy:  0.722
Epoch: 0490 c

Epoch: 0629 cost = 4.518537433Test accuracy:  0.736
Epoch: 0630 cost = 4.105770241Test accuracy:  0.778
Epoch: 0631 cost = 4.660855881Test accuracy:  0.727
Epoch: 0632 cost = 4.110546451Test accuracy:  0.713
Epoch: 0633 cost = 3.568261721Test accuracy:  0.787
Epoch: 0634 cost = 2.704252250Test accuracy:  0.731
Epoch: 0635 cost = 4.629300348Test accuracy:  0.593
Epoch: 0636 cost = 2.351877311Test accuracy:  0.704
Epoch: 0637 cost = 1.031913184Test accuracy:  0.741
Epoch: 0638 cost = 1.369040626Test accuracy:  0.704
Epoch: 0639 cost = 1.812452893Test accuracy:  0.722
Epoch: 0640 cost = 1.820453890Test accuracy:  0.708
Epoch: 0641 cost = 0.845167086Test accuracy:  0.708
Epoch: 0642 cost = 0.712056035Test accuracy:  0.741
Epoch: 0643 cost = 0.700994193Test accuracy:  0.727
Epoch: 0644 cost = 1.017918032Test accuracy:  0.713
Epoch: 0645 cost = 0.877385620Test accuracy:  0.741
Epoch: 0646 cost = 1.202935893Test accuracy:  0.741
Epoch: 0647 cost = 0.924669146Test accuracy:  0.769
Epoch: 0648 

...

Epoch: 3946 cost = 0.000002222Test accuracy:  0.671
Epoch: 3947 cost = 0.000002200Test accuracy:  0.671
Epoch: 3948 cost = 0.000002173Test accuracy:  0.671
Epoch: 3949 cost = 0.000002151Test accuracy:  0.671
Epoch: 3950 cost = 0.000002124Test accuracy:  0.671
Epoch: 3951 cost = 0.000002097Test accuracy:  0.671
Epoch: 3952 cost = 0.000002097Test accuracy:  0.671
Epoch: 3953 cost = 0.000002048Test accuracy:  0.671
Epoch: 3954 cost = 0.000002059Test accuracy:  0.671
Epoch: 3955 cost = 0.000002016Test accuracy:  0.671
Epoch: 3956 cost = 0.000001999Test accuracy:  0.671
Epoch: 3957 cost = 0.000001978Test accuracy:  0.671
Epoch: 3958 cost = 0.000001945Test accuracy:  0.671
Epoch: 3959 cost = 0.000001918Test accuracy:  0.671
Epoch: 3960 cost = 0.000001886Test accuracy:  0.671
Epoch: 3961 cost = 0.000001880Test accuracy:  0.671
Epoch: 3962 cost = 0.000001853Test accuracy:  0.671
Epoch: 3963 cost = 0.000001821Test accuracy:  0.671
Epoch: 3964 cost = 0.000001794Test accuracy:  0.671
Epoch: 3965 

Epoch: 4104 cost = 0.000000222Test accuracy:  0.671
Epoch: 4105 cost = 0.000000206Test accuracy:  0.671
Epoch: 4106 cost = 0.000000222Test accuracy:  0.671
Epoch: 4107 cost = 0.000000217Test accuracy:  0.671
Epoch: 4108 cost = 0.000000211Test accuracy:  0.671
Epoch: 4109 cost = 0.000000206Test accuracy:  0.671
Epoch: 4110 cost = 0.000000195Test accuracy:  0.671
Epoch: 4111 cost = 0.000000206Test accuracy:  0.671
Epoch: 4112 cost = 0.000000206Test accuracy:  0.671
Epoch: 4113 cost = 0.000000206Test accuracy:  0.667
Epoch: 4114 cost = 0.000000190Test accuracy:  0.671
Epoch: 4115 cost = 0.000000190Test accuracy:  0.671
Epoch: 4116 cost = 0.000000179Test accuracy:  0.671
Epoch: 4117 cost = 0.000000200Test accuracy:  0.671
Epoch: 4118 cost = 0.000000179Test accuracy:  0.671
Epoch: 4119 cost = 0.000000168Test accuracy:  0.671
Epoch: 4120 cost = 0.000000168Test accuracy:  0.671
Epoch: 4121 cost = 0.000000173Test accuracy:  0.667
Epoch: 4122 cost = 0.000000141Test accuracy:  0.671
Epoch: 4123 

Epoch: 4260 cost = 12.704641906Test accuracy:  0.532
Epoch: 4261 cost = 12.560797193Test accuracy:  0.551
Epoch: 4262 cost = 12.762106440Test accuracy:  0.556
Epoch: 4263 cost = 15.896517537Test accuracy:  0.537
Epoch: 4264 cost = 16.923405907Test accuracy:  0.532
Epoch: 4265 cost = 17.486808343Test accuracy:  0.574
Epoch: 4266 cost = 22.503446232Test accuracy:  0.542
Epoch: 4267 cost = 16.229176825Test accuracy:  0.593
Epoch: 4268 cost = 15.021817814Test accuracy:  0.56
Epoch: 4269 cost = 14.244020202Test accuracy:  0.556
Epoch: 4270 cost = 14.497239633Test accuracy:  0.569
Epoch: 4271 cost = 14.516377449Test accuracy:  0.579
Epoch: 4272 cost = 13.440797156Test accuracy:  0.542
Epoch: 4273 cost = 13.101296186Test accuracy:  0.574
Epoch: 4274 cost = 13.599162535Test accuracy:  0.546
Epoch: 4275 cost = 12.458266952Test accuracy:  0.514
Epoch: 4276 cost = 13.018344749Test accuracy:  0.602
Epoch: 4277 cost = 14.893654477Test accuracy:  0.574
Epoch: 4278 cost = 14.680198366Test accuracy:  

Epoch: 4416 cost = 6.547635219Test accuracy:  0.556
Epoch: 4417 cost = 6.364864523Test accuracy:  0.542
Epoch: 4418 cost = 6.614354795Test accuracy:  0.569
Epoch: 4419 cost = 7.001910990Test accuracy:  0.597
Epoch: 4420 cost = 5.912127592Test accuracy:  0.606
Epoch: 4421 cost = 5.527869734Test accuracy:  0.588
Epoch: 4422 cost = 6.221748786Test accuracy:  0.597
Epoch: 4423 cost = 7.043549017Test accuracy:  0.588
Epoch: 4424 cost = 9.452249765Test accuracy:  0.574
Epoch: 4425 cost = 10.078533129Test accuracy:  0.5
Epoch: 4426 cost = 5.947125988Test accuracy:  0.514
Epoch: 4427 cost = 6.266294501Test accuracy:  0.611
Epoch: 4428 cost = 5.863315701Test accuracy:  0.569
Epoch: 4429 cost = 6.303678870Test accuracy:  0.537
Epoch: 4430 cost = 7.003727198Test accuracy:  0.569
Epoch: 4431 cost = 6.444972808Test accuracy:  0.625
Epoch: 4432 cost = 6.679530989Test accuracy:  0.56
Epoch: 4433 cost = 5.676539697Test accuracy:  0.556
Epoch: 4434 cost = 5.862901406Test accuracy:  0.528
Epoch: 4435 co

KeyboardInterrupt: 

In [ ]:
m1.graph_setting()
m1.restore_graph(3)

In [ ]:
wav_file_path_test = 'C://slice_wav_data/testing/'
test_file_list = os.listdir(wav_file_path_test)
print(test_file_list)

In [ ]:
for file in test_file_list:
    m1.predict(wav_file_path_test + file)